In [9]:
import cv2
import dlib
import face_recognition
import numpy as np

# Initialize lists to hold known face encodings and names
known_face_encodings = []
known_face_names = []

# Load the images of known persons and get face encodings
known_person1_image = face_recognition.load_image_file("image1.jpg")
known_person2_image = face_recognition.load_image_file("image2.jpg")
known_person3_image = face_recognition.load_image_file("image3.jpg")
known_person4_image = face_recognition.load_image_file("image4.jpg")
known_person1_encoding = face_recognition.face_encodings(known_person1_image)[0]
known_person2_encoding = face_recognition.face_encodings(known_person2_image)[0]
known_person3_encoding = face_recognition.face_encodings(known_person3_image)[0]
known_person4_encoding = face_recognition.face_encodings(known_person4_image)[0]

# Append the encodings and names to the lists
known_face_encodings.extend([known_person1_encoding, known_person2_encoding, known_person3_encoding, known_person4_encoding])
known_face_names.extend(["Person 1", "Person 2", "Person 3", "Person 4"])

# Initialize video capture 
video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Load the smile cascade classifier
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# Initialize dlib's face detector and shape predictor for eye detection
detector = dlib.get_frontal_face_detector()
predictor_path = "shape_predictor_68_face_landmarks.dat"  
predictor = dlib.shape_predictor(predictor_path)

def detect_blink(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([(p.x, p.y) for p in landmarks.parts()])
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]
        
        left_eye_ratio = eye_aspect_ratio(left_eye)
        right_eye_ratio = eye_aspect_ratio(right_eye)
        
       
        blink_threshold = 0.3
        return left_eye_ratio < blink_threshold or right_eye_ratio < blink_threshold
    return False

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

while True:
    if video_capture.isOpened():
        ret, frame = video_capture.read()
        
        if not ret or frame is None:
            print("Failed to capture image or empty frame.")
            break
        
        # Convert image to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces using dlib
        faces = detector(gray)
        num = 0
        
        if len(faces) > 0:
            face_locations = face_recognition.face_locations(frame)
            face_encodings = face_recognition.face_encodings(frame, face_locations)
            
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                
                # Draw rectangle around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            

                roi_gray = gray[top:bottom, left:right]
                roi_color = frame[top:bottom, left:right]
                
                # Detect smiles within the face region
                smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
                smile_detected = "No smile detected"
                
                if len(smiles) > 0:
                    for (sx, sy, sw, sh) in smiles:
                        cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (255, 0, 0), 2)
                    smile_detected = "Smiling face detected"
                
                # Check for blink detection
                blink_detected = "Possible Fake Face"
                if detect_blink(frame[top:bottom, left:right]):
                    blink_detected = "Face is Live"
                
                # Display synchronized text information
                info_position = 20 + num * 60
                cv2.rectangle(frame, (10, info_position - 20), (300, info_position + 40), (0, 0, 0), -1)  # Background for text
                cv2.putText(frame, f"Face {num + 1}: {name}", (15, info_position), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, smile_detected, (15, info_position + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, blink_detected, (15, info_position + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
              
                num += 1
                cv2.putText(frame, "FACE number " + str(num), (left + 5, top - 10), cv2.FONT_HERSHEY_COMPLEX, 0.75, (0, 255, 0), 2)
             
          
        else:
            # Display no face found message
            cv2.rectangle(frame, (50, 10), (700, 100), (0, 0, 0), -1)
            cv2.putText(frame, "HELLO USER, NO FACE FOUND", (50, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "PLEASE MOVE CAMERA TOWARDS YOUR FACE", (60, 70), cv2.FONT_HERSHEY_COMPLEX, 0.75, (0, 0, 255), 2)

        # Display the resulting frame
        cv2.imshow("Windowframe", frame)

        # Break loop if 'ESC' key is pressed
        if cv2.waitKey(1) == 27:
            break
    else:
        print("Failed to open video capture.")
        break

# Release the video capture and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
